In [0]:
!pip install PyMuPDF
!pip install openai==0.28

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 32.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 896.4 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Not uninstalling openai at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-0c50acec-d699-4bab-8023-c07206296a75
    Can't uninstall 'openai'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install openai tenacity

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import openai
import requests
import os
import PyPDF2
from bs4 import BeautifulSoup

# Azure OpenAI configuration
openai.api_type = "azure"
openai.api_base = "https://gen-ai-certification-open-ai.openai.azure.com/"
openai.api_version = "2024-08-01-preview"
openai.api_key = "43kYkbFzKypJBbZVFb5r2BwTaW1TKb6gVdJ2Ry13B6nncjRXx7QjJQQJ99AKAC77bzfXJ3w3AAABACOGlQDL"

def search_pdfs(query, num_results=3):
    search_urls = [
        f"https://en.wikipedia.org/w/index.php?search={query}+filetype:pdf&title=Special:Search&profile=advanced&fulltext=1&ns0=1",
        f"https://scholar.google.com/scholar?q={query}+filetype:pdf",
        f"https://www.researchgate.net/search/publication?q={query}"
    ]
    
    pdf_urls = []
    for search_url in search_urls:
        response = requests.get(search_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        pdf_links = soup.find_all('a', href=lambda href: href and href.lower().endswith('.pdf'))
        pdf_urls.extend([link['href'] for link in pdf_links])
        if len(pdf_urls) >= num_results:
            break
    
    return pdf_urls[:num_results]

def download_pdf(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded {filename}")
        return True
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")
        return False

def extract_text_from_pdf(filename):
    text = ""
    try:
        with open(filename, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error reading {filename}: {str(e)}")
    return text

# Main execution
user_prompt = input("Enter your search prompt: ")
pdf_urls = search_pdfs(user_prompt)

dataset = []

for i, url in enumerate(pdf_urls):
    filename = f"document_{i+1}.pdf"
    
    if download_pdf(url, filename):
        text = extract_text_from_pdf(filename)
        dataset.append({
            "document_id": i + 1,
            "text": text,
        })
        
        print(f"\nContent of {filename}:")
        print(text[:500])  # Print first 500 characters
        print("...")
        
        os.remove(filename)

print(f"\nTotal documents processed: {len(dataset)}")

## gives information correctly mostly related to techonoloigcal content

Enter your search prompt:  what is bluettoth

Successfully downloaded document_1.pdf

Content of document_1.pdf:
1 
  
 
Bluetooth  
 
     
 
Author: Inigo Puy 
Mtknr: 230345 
Course: E-Business 
Professor: Eduard Heindl 
Date: 05/05/08 

2 
  
 
 
 
  
 
 
 
In this page I declare that this work cites all sources used to make it and that this 
work was done by me. I declare that all the work has been redact ed by me except the 
paragraphs that are written in cursive characters and which have t he exact link to the 
place they were taken from. All the imagines and diagrams have t hey own links to the 
sou
...
Successfully downloaded document_2.pdf

Content of document_2.pdf:
1 
  
 
Bluetooth  
 
     
 
Author: Inigo Puy 
Mtknr: 230345 
Course: E-Business 
Professor: Eduard Heindl 
Date: 05/05/08 

2 
  
 
 
 
  
 
 
 
In this page I declare that this work cites all sources used to make it and that this 
work was done by me. I declare that all the work has been redact ed by me except the 
paragraphs that are written in cursive c